<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Iterator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterator / Итератор
Предоставляет способ последовательного доступа ко всем элементам составного объекта, не раскрывая его внутреннего представления.
Используется:
- для доступа к содержимому агрегированных объектов без раскрытия их внутреннего представления.
- для поддержки нескольких активных обходов одного и того же агрегированного объекта.
- для предоставления единообразного интерфейса с целью обхода различных агрегированных структур (т.е. для поддержки полиморфной итерации)

## Пример реализации 1
Построим прямой и обратный итераторы, применим их на списках людей и домов.

In [1]:
class Abstract_List:
    def create_iterator(self,iterator_cls):
        return iterator_cls(self)
    def count(self):
        pass
    def append(self,item):
        pass
    def remove(self,item):
        pass

class Abstract_Iterator:
    def __init__(self,list_obj):
        self._index = None
        self._list = list_obj
    def first(self):
        pass
    def next(self):
        pass
    def is_done(self):
        pass
    def current_item(self):
        pass

class Person_List(Abstract_List):
    def __init__(self):
        self._list = []
    def count(self):
        return len(self._list)
    def append(self,item):
        self._list.append(item)
    def remove(self,item):
        self._list.pop(self._list.index(item))
    def __getitem__(self, key):
        return self._list[key]
    def show(self):
        for i in self._list:
            print(i)

class Straight_Iterator(Abstract_Iterator):
    def first(self):
        self._index = 0
    def next(self):
        if not self.is_done():
            self._index += 1
        else:
            raise Exception('Out of range error!')
    def is_done(self):
        if self._list.count() <= self._index:
            return True
        else:
            return False
    def current_item(self):
        return self._list[self._index]

class Backward_Iterator(Abstract_Iterator):
    def first(self):
        self._index = self._list.count()-1
    def next(self):
        if not self.is_done():
            self._index -= 1
        else:
            raise Exception('Out of range error!')
    def is_done(self):
        if self._index < 0:
            return True
        else:
            return False
    def current_item(self):
        return self._list[self._index]

class House:
    def __init__(self, street, house):
        self.street = street
        self.house = house

    def __str__(self):
         return ' '.join([self.street, self.house])

class House_dict(Abstract_List):
    def __init__(self):
        self._list = []
    def count(self):
        return len(self._list)
    def append(self,item : House):
        self._list.append(item)
    def remove(self,item):
        self._list.pop(item)
    def __getitem__(self, key):
        return self._list[key]
    def show(self):
        for i in self._list:
            print(i)

mylist = Person_List()
mylist.append('Ivan Ivanov')
mylist.append('Sergei Sergeev')
mylist.append('Prohor Prohorov')
mylist.append('Anna Sergeeva')
print('Наш начальный список людей:')
mylist.show()
print('Людей в списке: ',mylist.count())
print('Удаляем Ivan Ivanov, теперь список выглядит так: ')
mylist.remove('Ivan Ivanov')
mylist.show()
print('Запускаем прямой итератор по списку людей: ')
iter1 = mylist.create_iterator(Straight_Iterator)
iter1.first()
while not iter1.is_done():
    print(iter1.current_item())
    iter1.next()

print('Запускаем обратный итератор по списку людей: ')
iter2 = mylist.create_iterator(Backward_Iterator)
iter2.first()
while not iter2.is_done():
    print(iter2.current_item())
    iter2.next()

print('Запускаем одновременно прямой и обратный итераторы по списку людей: ')
iter3 = mylist.create_iterator(Straight_Iterator)
iter4 = mylist.create_iterator(Backward_Iterator)
iter3.first()
iter4.first()
while not (iter3.is_done() and iter4.is_done()):
    print('->: '+ iter3.current_item(), ' - ', '<-: ' + iter4.current_item())
    iter3.next()
    iter4.next()


Наш начальный список людей:
Ivan Ivanov
Sergei Sergeev
Prohor Prohorov
Anna Sergeeva
Людей в списке:  4
Удаляем Ivan Ivanov, теперь список выглядит так: 
Sergei Sergeev
Prohor Prohorov
Anna Sergeeva
Запускаем прямой итератор по списку людей: 
Sergei Sergeev
Prohor Prohorov
Anna Sergeeva
Запускаем обратный итератор по списку людей: 
Anna Sergeeva
Prohor Prohorov
Sergei Sergeev
Запускаем одновременно прямой и обратный итераторы по списку людей: 
->: Sergei Sergeev  -  <-: Anna Sergeeva
->: Prohor Prohorov  -  <-: Prohor Prohorov
->: Anna Sergeeva  -  <-: Sergei Sergeev


# Пример реализации 2
С помощью перегрузки магических методов можно построить более изящный синтаксически вариант:

In [2]:
class Abstract_iterator:
    def __init__(self,data):
        self._data = data
    def __getitem__(self, key):
        return self._data[key]
    def __iter__(self):
        pass
    def __next__(self):
        pass

class Straight_Iterator(Abstract_iterator):
    def __iter__(self):
        self.ix = 0
        return self

    def __next__(self):
        if self.ix >= len(self._data):
            raise StopIteration
        item = self._data[self.ix]
        self.ix += 1
        return item

class Backward_iterator(Abstract_iterator):
    def __iter__(self):
        self.ix = len(self._data)-1
        return self

    def __next__(self):
        if self.ix < 0:
            raise StopIteration
        item = self._data[self.ix]
        self.ix -=1
        return item

class Abstract_List:
    def __init__(self,data):
        self._data = data

    def __getitem__(self, item):
        return self._data[item]

    def __len__(self):
        return len(self._data)

    def create_iterator(self,iterator_cls):
        return iterator_cls(self)

    def __str__(self):
        return ', '.join(str(i) for i in self._data)

class SomeList(Abstract_List):
  pass

if __name__=='__main__':
    print('Инициализируем список')
    x = SomeList([1,2,3,4,5])
    print(x)
    print('Запускаем прямой итератор')
    i1 = x.create_iterator(Straight_Iterator)
    for k in i1:
        print(k)
    print('Запускаем обратный итератор')
    i2 = x.create_iterator(Backward_iterator)
    for k in i2:
        print(k)

Инициализируем список
1, 2, 3, 4, 5
Запускаем прямой итератор
1
2
3
4
5
Запускаем обратный итератор
5
4
3
2
1
